# COS-Halos Database (v1.1)

This Notebook is intended to record updates to the COS-Halos database.

Version History:

 * v1.0 -- 09 Feb 2016 by JXP
 * v1.1 -- 13 Apr 2016 by JXP

## NOTE:  The actual updating should use the update_cos_halos method in cos_halos.py

In [1]:
# imports
import pdb
from pyigm.cgm.cos_halos import COSHalos
from pyigm.cgm import cos_halos as pch

## Update the Megastructure

J0910+1014_34_46

has coverage of 5 SiII transitions yet the 
megastructure implies only 1:

IDL> help, megastruct[ii].ion[14,2], /str

    LOGNION         DOUBLE           12.957801
    SIGLOGNION      DOUBLE         0.081717335
    NFLG            LONG                 1
    NTRANS          LONG                 1
    BVAL            DOUBLE           0.0000000
    SIGBVAL         DOUBLE           0.0000000
    TRANS           STRUCT    -> TMPTRANS Array[16]


## Load FITS files from Megastructure

Generated on Dec 7, 2014 by JXP

This step will update the f-values (several lines were
using the old Morton 1991 reference) and associated column densities.

The synthesize_colm() method of AbsComponent is used for final columns.
These match the original to within 10% in all cases.

In [4]:
reload(pch)
cos_halos = pch.COSHalos()
cos_halos.load_mega()#test=True)

cos_halos: Reading /Users/xavier/Dropbox/COS-Halos-Data//Summary/J0226+0015.268_22.fits.gz
Skipping column contribution from 1025.72 as NG for a line; flg=2
Skipping column contribution from 972.537 as NG for a line; flg=2
cos_halos: Reading /Users/xavier/Dropbox/COS-Halos-Data//Summary/J0401-0540.67_24.fits.gz
Skipping column contribution from 1025.72 as NG for a line; flg=2
cos_halos: Reading /Users/xavier/Dropbox/COS-Halos-Data//Summary/J0803+4332.306_20.fits.gz
cos_halos: Reading /Users/xavier/Dropbox/COS-Halos-Data//Summary/J0910+1014.242_34.fits.gz
Skipping column contribution from 949.743 as NG for a line; flg=2
Skipping column contribution from 937.803 as NG for a line; flg=13
Skipping column contribution from 930.748 as NG for a line; flg=13
Skipping column contribution from 926.226 as NG for a line; flg=15
Skipping column contribution from 1304.37 as NG for a line; flg=10
cos_halos: Reading /Users/xavier/Dropbox/COS-Halos-Data//Summary/J0910+1014.34_46.fits.gz
Skipping column

/Users/xavier/local/Python/pyigm/pyigm/cgm/cos_halos.py:255: UserWarning: This module will be DEPRECATED
  warnings.warn("This module will be DEPRECATED")
/Users/xavier/local/Python/linetools/linetools/analysis/absline.py:125: RuntimeWarning: invalid value encountered in double_scalars
  lgvar = ((1.0 / (np.log(10.0)*N))**2)*sig_N**2



cos_halos: Reading /Users/xavier/Dropbox/COS-Halos-Data//Summary/J1435+3604.126_21.fits.gz
Skipping column contribution from 949.743 as NG for a line; flg=2
Skipping column contribution from 930.748 as NG for a line; flg=2
Skipping column contribution from 926.226 as NG for a line; flg=2
cos_halos: Reading /Users/xavier/Dropbox/COS-Halos-Data//Summary/J1435+3604.68_12.fits.gz
Skipping column contribution from 949.743 as NG for a line; flg=13
New colm for (8,1) and sys J1435+3604_68_12 is 0.854291 different from old
cos_halos: Reading /Users/xavier/Dropbox/COS-Halos-Data//Summary/J1437+5045.317_38.fits.gz
Skipping column contribution from 949.743 as NG for a line; flg=4
Skipping column contribution from 1302.17 as NG for a line; flg=13
New colm for (8,1) and sys J1437+5045_317_38 is -14.6919 different from old
Skipping column contribution from 933.378 as NG for a line; flg=13
New colm for (16,6) and sys J1437+5045_317_38 is 0.255336 different from old
cos_halos: Reading /Users/xavier/D

/Users/xavier/local/Python/pyigm/pyigm/cgm/cos_halos.py:212: RuntimeWarning: invalid value encountered in double_scalars
  if np.abs(comp.logN - float(iont['CLM'][0])) > 0.15:


## Write to JSON tarball

# FIX sig_logN for limits?

In [5]:
tarfil = cos_halos.cdir+'/cos-halos_systems.v1.0.tar.gz'

In [6]:
cos_halos.write_survey(tarfil)

Wrote: /Users/xavier/Dropbox/COS-Halos-Data//cos-halos_systems.v1.0.tar.gz


/Users/xavier/local/Python/pyigm/pyigm/cgm/cgmsurvey.py:70: UserWarning: Modify to write directly to tar file
  warnings.warn("Modify to write directly to tar file")


----

# Generating v1.1

## Update $N_{\rm HI}$ measurements (Prochaska+16)

### Load JSON files

In [7]:
from pyigm.cgm import cos_halos as pch
import pdb

In [8]:
reload(pch)
cos_halos_v10 = pch.COSHalos()
cos_halos_v10.load_sys(tfile=cos_halos_v10.cdir+'/cos-halos_systems.v1.0.tar.gz')

Be patient, using /Users/xavier/Dropbox/COS-Halos-Data//cos-halos_systems.v1.0.tar.gz to load COS-Halos
Skipping a likely folder: CGM_JSON


### Load NHI measurements

In [9]:
import json
LLS_file = cos_halos_v10.cdir + '/Updates/COS_Halos_LLS.json'  # Generated in paper/COS-Halos/Patchup/Analysis
with open(LLS_file) as json_file:
    fdict = json.load(json_file)
#fdict

### Loop on systems

In [10]:
names = cos_halos_v10.name

In [11]:
for key in fdict.keys():
    # Match
    mt = np.where(names == key)[0]
    if len(mt) != 1:
        raise ValueError("No match?!")
    # Fill in 
    if fdict[key]['flag_NHI'] == 1:
        cos_halos_v10.cgm_abs[mt[0]].igm_sys.NHI = fdict[key]['fit_NHI'][0]
        cos_halos_v10.cgm_abs[mt[0]].igm_sys.sig_NHI = (fdict[key]['fit_NHI'][2]-fdict[key]['fit_NHI'][1])/2.
        cos_halos_v10.cgm_abs[mt[0]].igm_sys.flag_NHI = 1
    elif fdict[key]['flag_NHI'] == 2:
        cos_halos_v10.cgm_abs[mt[0]].igm_sys.NHI = fdict[key]['fit_NHI'][1]
        cos_halos_v10.cgm_abs[mt[0]].igm_sys.flag_NHI = 2
    elif fdict[key]['flag_NHI'] == 3:
        cos_halos_v10.cgm_abs[mt[0]].igm_sys.NHI = fdict[key]['fit_NHI'][2]
        cos_halos_v10.cgm_abs[mt[0]].igm_sys.flag_NHI = 3

## Modify a few column density measurements (Prochaska+16)

### Load bad ions

In [12]:
import yaml
filename = cos_halos_v10.cdir + '/Updates/COS_Halos_ions_updates.yaml'
with open(filename, 'r') as infile:
        up_ion_data = yaml.load(infile)

In [13]:
up_ion_data

{'J0910+1014_34_46': {'ion': {'7,2': {'flag_N': 3}}},
 'J0928+6025_110_35': {'trans': {'FeIII 1122': {'flag_N': 1}}},
 'J0943+0531_227_19': {'ion': {'6,2': {'flag_N': 3}, '7,2': {'flag_N': 3}}},
 'J1016+4706_274_6': {'trans': {'FeII 1144': {'flag_N': 3}}},
 'J1233+4758_94_38': {'ion': {'6,3': {'flag_N': 0}}},
 'J1342-0053_157_10': {'trans': {'OI 971': {'flag_N': 3}}},
 'J1435+3604_68_12': {'trans': {'OI 971': {'flag_N': 3}}},
 'J1619+3342_113_40': {'ion': {'6,2': {'flag_N': 3}}},
 'J2345-0059_356_12': {'trans': {'NII 1083': {'flag_N': 3},
   'SiIII 1206': {'flag_N': 0}}}}

In [14]:
for key in up_ion_data.keys():
    # Match
    mt = np.where(names == key)[0]
    if len(mt) != 1:
        raise ValueError("No match?!")
    igm_sys = cos_halos_v10.cgm_abs[mt[0]].igm_sys
    # Fill in 
    for mod_type in up_ion_data[key].keys():
        if mod_type == 'ion':
            for ionkey in up_ion_data[key][mod_type].keys():
                Zion = tuple([int(ii) for ii in ionkey.split(',')])
                #
                Zicomp = [comp.Zion for comp in igm_sys._components]
                mtZi = Zicomp.index(Zion)
                # Set
                for att_key in up_ion_data[key][mod_type][ionkey].keys():
                    if att_key == 'flag_N':
                        #cos_halos_v10.cgm_abs[mt[0]].igm_sys._components[mtZi].flag_N = up_ion_data[key][mod_type][ionkey][att_key]
                        igm_sys._components[mtZi].flag_N = up_ion_data[key][mod_type][ionkey][att_key]
                    else: 
                        raise ValueError("Bad key for attribute")
                print(cos_halos_v10.cgm_abs[mt[0]].igm_sys._components[mtZi])
        elif mod_type == 'trans':
            for transkey in up_ion_data[key][mod_type].keys():
                # Update AbsLine
                lines = igm_sys.list_of_abslines()
                trans = [iline.name for iline in lines]
                aline = lines[trans.index(transkey)]
                comp = igm_sys.get_comp_from_absline(aline)  # Grab it now before it changes
                if att_key == 'flag_N':
                    aline.attrib['flag_N'] = up_ion_data[key][mod_type][transkey][att_key]
                # Remake component
                try:
                    comp.synthesize_colm(overwrite=True)
                except ValueError:
                    pdb.set_trace()
        else: 
            raise ValueError("Bad mod_type")

<AbsComponent: 12:33:35.1 +47:58:00.5, Name=CIII_z0.22210, Zion=(6,3), Ej=0 1 / cm, z=0.222103, vlim=-84 km / s,80 km / s>
<AbsComponent: 09:43:31.6 +05:31:31.5, Name=NII_z0.35295, Zion=(7,2), Ej=0 1 / cm, z=0.352952, vlim=-50 km / s,450 km / s, logN=14.711, sig_N=8.36472e-14, flag_N=3>
<AbsComponent: 09:43:31.6 +05:31:31.5, Name=CII_z0.35295, Zion=(6,2), Ej=0 1 / cm, z=0.352952, vlim=-15 km / s,547 km / s, logN=14.4414, sig_N=0.0921919, flag_N=3>
<AbsComponent: 16:19:16.5 +33:42:38.4, Name=CII_z0.14137, Zion=(6,2), Ej=0 1 / cm, z=0.141367, vlim=-160 km / s,55 km / s, logN=14.3184, sig_N=0.0391796, flag_N=3>
<AbsComponent: 09:10:29.8 +10:14:13.6, Name=NII_z0.14274, Zion=(7,2), Ej=0 1 / cm, z=0.142738, vlim=-212 km / s,40 km / s, logN=14.3247, sig_N=0.135632, flag_N=3>


### Write

In [15]:
tarfil = cos_halos_v10.cdir+'/cos-halos_systems.v1.1.tar.gz'

In [16]:
cos_halos_v10.write_survey(tarfil)

Wrote: /Users/xavier/Dropbox/COS-Halos-Data//cos-halos_systems.v1.1.tar.gz


### Tests

In [15]:
cgm = cos_halos_v10[('J0928+6025','110_35')]

In [17]:
cgm.igm_sys.get_absline('SiIII 1206').attrib

{u'EW': <Quantity 0.63364048743187 Angstrom>,
 u'N': <Quantity 60611606990686.49 1 / cm2>,
 u'b': <Quantity 0.0 km / s>,
 u'coord': <SkyCoord (ICRS): (ra, dec) in deg
     (142.15833333, 60.4225)>,
 u'flag_EW': 2,
 u'flag_N': 2,
 u'logN': 13.782555798577503,
 u'sig_EW': <Quantity 0.05809608690608618 Angstrom>,
 u'sig_N': <Quantity 5924195705267.877 1 / cm2>,
 u'sig_b': <Quantity 0.0 km / s>,
 u'sig_logN': 0.04244806617498398,
 u'sig_v': <Quantity 0.0 km / s>,
 u'sig_z': 0.0,
 u'v': <Quantity 0.0 km / s>,
 u'z': 0.1540004312992096}